# Complete Deep Learning Optimization

Perform optimization accfrom tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from keras.callbacks import EarlyStoppingording to the following:

| | | | 
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.3, portfolio optimization as done in https://www.quantconnect.com/terminal/processCache/?request=embedded_backtest_4ebbe01bfea8c5ae6f98fcda38a50b1c.html

In [81]:
from tensorflow.keras.layers import LSTM, Flatten, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import pickle
import sklearn
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from keras.callbacks import EarlyStopping

## Set up the Model 

I *think* I will need to reinitialize this class with the correct "data" for each estimation.

In [222]:
## Read the Data
file_name = "PortfolioOptimizationData.pkl"

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

file_name_os = "PortfolioOptimizationData_Zeros.pkl"

open_file_os = open(file_name_os, "rb")
loaded_list_os = pickle.load(open_file_os)
open_file_os.close()

In [223]:
data_n4_corr = loaded_list[0]
data_n8_corr = loaded_list[1]
data_n16_corr = loaded_list[2]

data_n4_ind = loaded_list[3]
data_n8_ind = loaded_list[4]
data_n16_ind = loaded_list[5]

data_n4_neg = loaded_list[6]
data_n8_neg = loaded_list[7]
data_n16_neg = loaded_list[8]

data_n4_corr_os = loaded_list_os[0]
data_n8_corr_os = loaded_list_os[1]
data_n16_corr_os = loaded_list_os[2]

data_n4_ind_os = loaded_list_os[3]
data_n8_ind_os = loaded_list_os[4]
data_n16_ind_os = loaded_list_os[5]

data_n4_neg_os = loaded_list_os[6]
data_n8_neg_os = loaded_list_os[7]
data_n16_neg_os = loaded_list_os[8]

In [237]:
# temporarily surpress output
#sys.stdout = open(os.devnull, "w")
#sys.stderr = open(os.devnull, "w")
# to return it
sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

## Perform the optimization for positively simulated data
Correlation from 0.8-0.9
Training Data for Positively correlated data

In [169]:
# No zeros
n_assets = 4
n_obs = 1000

In [178]:
weight_n4_corr = np.zeros((100,n_assets))
return_n4_corr = np.zeros((100,n_assets))
risks_n4_corr = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_corr[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_corr[i,:] = weights
    for j in range(n_assets):
        return_n4_corr[i,j] = np.sum(weight_n4_corr[i,j] * data_n4_corr[i][j])
        risks_n4_corr[i,j] = weight_n4_corr[i,j] * np.diagonal(np.cov(data_n4_corr[i].T))[j] * weight_n4_corr[i,j]
    

In [238]:
print('weights (mean) = '+str(np.round(np.mean(weight_n4_corr, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(return_n4_corr, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(risks_n4_corr, axis=0),3)))

In [72]:
# Zeros 

In [180]:
weight_n4_corr_os = np.zeros((100,n_assets))
return_n4_corr_os = np.zeros((100,n_assets))
risks_n4_corr_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_corr_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_corr_os[i,:] = weights
    for j in range(n_assets):
        return_n4_corr_os[i,j] = np.sum(weight_n4_corr_os[i,j] * data_n4_corr_os[i][j])
        risks_n4_corr_os[i,j] = weight_n4_corr_os[i,j] * np.diagonal(np.cov(data_n4_corr_os[i].T))[j] * weight_n4_corr_os[i,j]

In [ ]:
# n = 8 assets

In [182]:
n_assets = 8
n_obs = 1000

In [ ]:
# no zeros

In [183]:
weight_n8_corr = np.zeros((100,n_assets))
return_n8_corr = np.zeros((100,n_assets))
risks_n8_corr = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_corr[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_corr[i,:] = weights
    for j in range(n_assets):
        return_n8_corr[i,j] = np.sum(weight_n8_corr[i,j] * data_n8_corr[i][j])
        risks_n8_corr[i,j] = weight_n8_corr[i,j] * np.diagonal(np.cov(data_n8_corr[i].T))[j] * weight_n8_corr[i,j]

In [ ]:
# zeros

In [184]:
weight_n8_corr_os = np.zeros((100,n_assets))
return_n8_corr_os = np.zeros((100,n_assets))
risks_n8_corr_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n8_corr_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_corr_os[i,:] = weights
    for j in range(n_assets):
        return_n8_corr_os[i,j] = np.sum(weight_n8_corr_os[i,j] * data_n8_corr_os[i][j])
        risks_n8_corr_os[i,j] = weight_n8_corr_os[i,j] * np.diagonal(np.cov(data_n8_corr_os[i].T))[j] * weight_n8_corr_os[i,j]

In [181]:
# n = 16

In [196]:
n_assets = 16
n_obs = 1000

In [198]:
# no zeros
data_n16_corr.shape

(100, 1000, 4)

In [201]:
weight_n16_corr = np.zeros((100,n_assets))
return_n16_corr = np.zeros((100,n_assets))
risks_n16_corr = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_corr[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_corr[i,:] = weights
    for j in range(n_assets):
        return_n16_corr[i,j] = np.sum(weight_n16_corr[i,j] * data_n16_corr[i][j])
        risks_n16_corr[i,j] = weight_n16_corr[i,j] * np.diagonal(np.cov(data_n16_corr[i].T))[j] * weight_n16_corr[i,j]

In [ ]:
# zeros

In [205]:
weight_n16_corr_os = np.zeros((100,n_assets))
return_n16_corr_os = np.zeros((100,n_assets))
risks_n16_corr_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_corr_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_corr_os[i,:] = weights
    for j in range(n_assets):
        return_n16_corr_os[i,j] = np.sum(weight_n16_corr_os[i,j] * data_n16_corr_os[i][j])
        risks_n16_corr_os[i,j] = weight_n16_corr_os[i,j] * np.diagonal(np.cov(data_n16_corr_os[i].T))[j] * weight_n16_corr_os[i,j]

## Independent

In [ ]:
# n = 4 assets

In [206]:
n_assets = 4
n_obs = 1000

In [ ]:
# no zeros

In [207]:
weight_n4_ind = np.zeros((100,n_assets))
return_n4_ind = np.zeros((100,n_assets))
risks_n4_ind = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_ind[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_ind[i,:] = weights
    for j in range(n_assets):
        return_n4_ind[i,j] = np.sum(weight_n4_ind[i,j] * data_n4_ind[i][j])
        risks_n4_ind[i,j] = weight_n4_ind[i,j] * np.diagonal(np.cov(data_n4_ind[i].T))[j] * weight_n4_ind[i,j]
    

In [ ]:
# zeros

In [208]:
weight_n4_ind_os = np.zeros((100,n_assets))
return_n4_ind_os = np.zeros((100,n_assets))
risks_n4_ind_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_ind_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_ind_os[i,:] = weights
    for j in range(n_assets):
        return_n4_ind_os[i,j] = np.sum(weight_n4_ind_os[i,j] * data_n4_ind_os[i][j])
        risks_n4_ind_os[i,j] = weight_n4_ind_os[i,j] * np.diagonal(np.cov(data_n4_ind_os[i].T))[j] * weight_n4_ind_os[i,j]
    

In [ ]:
# n = 8

In [209]:
n_assets = 8
n_obs = 1000

In [ ]:
# No zeros

In [210]:
weight_n8_ind = np.zeros((100,n_assets))
return_n8_ind = np.zeros((100,n_assets))
risks_n8_ind = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n8_ind[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_ind[i,:] = weights
    for j in range(n_assets):
        return_n8_ind[i,j] = np.sum(weight_n8_ind[i,j] * data_n8_ind[i][j])
        risks_n8_ind[i,j] = weight_n8_ind[i,j] * np.diagonal(np.cov(data_n8_ind[i].T))[j] * weight_n8_ind[i,j]
    

In [ ]:
# zeros

In [211]:
weight_n8_ind_os = np.zeros((100,n_assets))
return_n8_ind_os = np.zeros((100,n_assets))
risks_n8_ind_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n8_ind_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_ind_os[i,:] = weights
    for j in range(n_assets):
        return_n8_ind_os[i,j] = np.sum(weight_n8_ind_os[i,j] * data_n8_ind_os[i][j])
        risks_n8_ind_os[i,j] = weight_n8_ind_os[i,j] * np.diagonal(np.cov(data_n8_ind_os[i].T))[j] * weight_n8_ind_os[i,j]
    

In [ ]:
# n = 16

In [212]:
n_assets = 16
n_obs = 1000

In [ ]:
# no zeros

In [225]:
weight_n16_ind = np.zeros((100,n_assets))
return_n16_ind = np.zeros((100,n_assets))
risks_n16_ind = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_ind[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_ind[i,:] = weights
    for j in range(n_assets):
        return_n16_ind[i,j] = np.sum(weight_n16_ind[i,j] * data_n16_ind[i][j])
        risks_n16_ind[i,j] = weight_n16_ind[i,j] * np.diagonal(np.cov(data_n16_ind[i].T))[j] * weight_n16_ind[i,j]

In [224]:
# zeros

(100, 1000, 16)

In [226]:
weight_n16_ind_os = np.zeros((100,n_assets))
return_n16_ind_os = np.zeros((100,n_assets))
risks_n16_ind_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_ind_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_ind_os[i,:] = weights
    for j in range(n_assets):
        return_n16_ind_os[i,j] = np.sum(weight_n16_ind_os[i,j] * data_n16_ind_os[i][j])
        risks_n16_ind_os[i,j] = weight_n16_ind_os[i,j] * np.diagonal(np.cov(data_n16_ind_os[i].T))[j] * weight_n16_ind_os[i,j]
    

## Negative Correlation

In [ ]:
# n = 4

In [227]:
n_assets = 4
n_obs = 1000

In [ ]:
# zeros

In [228]:
weight_n4_neg = np.zeros((100,n_assets))
return_n4_neg = np.zeros((100,n_assets))
risks_n4_neg = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_neg[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_neg[i,:] = weights
    for j in range(n_assets):
        return_n4_neg[i,j] = np.sum(weight_n4_neg[i,j] * data_n4_neg[i][j])
        risks_n4_neg[i,j] = weight_n4_neg[i,j] * np.diagonal(np.cov(data_n4_neg[i].T))[j] * weight_n4_neg[i,j]
    

In [ ]:
# zeros

In [229]:
weight_n4_neg_os = np.zeros((100,n_assets))
return_n4_neg_os = np.zeros((100,n_assets))
risks_n4_neg_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n4_neg_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n4_neg_os[i,:] = weights
    for j in range(n_assets):
        return_n4_neg_os[i,j] = np.sum(weight_n4_neg_os[i,j] * data_n4_neg_os[i][j])
        risks_n4_neg_os[i,j] = weight_n4_neg_os[i,j] * np.diagonal(np.cov(data_n4_neg_os[i].T))[j] * weight_n4_neg_os[i,j]
    

In [ ]:
# n = 8

In [230]:
n_assets = 8
n_obs = 1000

In [231]:
weight_n8_neg = np.zeros((100,n_assets))
return_n8_neg = np.zeros((100,n_assets))
risks_n8_neg = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n8_neg[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_neg[i,:] = weights
    for j in range(n_assets):
        return_n8_neg[i,j] = np.sum(weight_n8_neg[i,j] * data_n8_neg[i][j])
        risks_n8_neg[i,j] = weight_n8_neg[i,j] * np.diagonal(np.cov(data_n8_neg[i].T))[j] * weight_n8_neg[i,j]

In [ ]:
# zeros

In [232]:
weight_n8_neg_os = np.zeros((100,n_assets))
return_n8_neg_os = np.zeros((100,n_assets))
risks_n8_neg_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n8_neg_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n8_neg_os[i,:] = weights
    for j in range(n_assets):
        return_n8_neg_os[i,j] = np.sum(weight_n8_neg_os[i,j] * data_n8_neg_os[i][j])
        risks_n8_neg_os[i,j] = weight_n8_neg_os[i,j] * np.diagonal(np.cov(data_n8_neg_os[i].T))[j] * weight_n8_neg_os[i,j]

In [ ]:
# n = 16

In [233]:
n_assets = 16
n_obs = 1000

In [ ]:
# no zeros

In [234]:
weight_n16_neg = np.zeros((100,n_assets))
return_n16_neg = np.zeros((100,n_assets))
risks_n16_neg = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_neg[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_neg[i,:] = weights
    for j in range(n_assets):
        return_n16_neg[i,j] = np.sum(weight_n16_neg[i,j] * data_n16_neg[i][j])
        risks_n16_neg[i,j] = weight_n16_neg[i,j] * np.diagonal(np.cov(data_n16_neg[i].T))[j] * weight_n16_neg[i,j]

In [ ]:
# zeros

In [235]:
weight_n16_neg_os = np.zeros((100,n_assets))
return_n16_neg_os = np.zeros((100,n_assets))
risks_n16_neg_os = np.zeros((100,n_assets))
    
for i in range(100):
    data = data_n16_neg_os[i]
    
    class Model:
        def __init__(self):
            self.data = None
            self.model = None
        
            # self.callback = EarlyStopping(monitor='loss', min_delta=.1, patience = 10)
    
        def __build_model(self, input_shape, outputs):
            model = Sequential([
                LSTM(64, input_shape=input_shape, activation='relu'),
                Flatten(),
                Dense(outputs, activation='softmax')
            ])

            def sharpe_loss(_, y_pred):
                coeffs = tf.tile(y_pred, (self.data.shape[0], 1))
            
                portfolio_values = tf.reduce_sum(tf.multiply(coeffs, self.data), axis=1)
            
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula

                sharpe = K.mean(portfolio_returns) / K.std(portfolio_returns)
            
                # exp keeps relative ordering between positives and negatives
                #   since we want to maximize sharp, while gradient descent minimizes the loss
                #   we negate the Sharpe value
                return K.exp(-sharpe)
        
            model.compile(loss=sharpe_loss, optimizer='adam')
            return model
    
        def get_allocations(self, data):
        
        
            # data with returns
            data_w_ret = np.concatenate([ data.values[1:], data.pct_change().values[1:] ], axis=1)
        
            data = data.iloc[1:]
            self.data = tf.cast(tf.constant(data), float)
        
            if self.model is None:
                self.model = self.__build_model(data_w_ret.shape, len(data.columns))
        
            fit_predict_data = data_w_ret[np.newaxis,:]        
            self.model.fit(fit_predict_data, np.zeros((1, len(data.columns))), epochs=10, shuffle=False)
            return self.model.predict(fit_predict_data)[0]
    
    model = Model() 
    weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets)))
    weight_n16_neg_os[i,:] = weights
    for j in range(n_assets):
        return_n16_neg_os[i,j] = np.sum(weight_n16_neg_os[i,j] * data_n16_neg_os[i][j])
        risks_n16_neg_os[i,j] = weight_n16_neg_os[i,j] * np.diagonal(np.cov(data_n16_neg_os[i].T))[j] * weight_n16_neg_os[i,j]

## Save the results

In [263]:
DeepLearningResults_Weights = [weight_n4_corr, weight_n8_corr, weight_n16_corr, weight_n4_corr_os, 
                              weight_n8_corr_os, weight_n16_corr_os, weight_n4_ind, weight_n8_ind,
                              weight_n16_ind, weight_n4_ind_os, weight_n8_ind_os, weight_n16_ind_os,
                              weight_n4_neg, weight_n8_neg, weight_n16_neg, weight_n4_neg_os, 
                              weight_n8_neg_os, weight_n16_neg_os]

file_name = "DeepLearningResultsWeights.pkl"
open_file = open(file_name, "wb")
pickle.dump(DeepLearningResults_Weights, open_file)
open_file.close()

In [264]:
DeepLearningResults_Returns = [return_n4_corr, return_n8_corr, return_n16_corr, return_n4_corr_os, 
                              return_n8_corr_os, return_n16_corr_os, return_n4_ind, return_n8_ind,
                              return_n16_ind, return_n4_ind_os, return_n8_ind_os, return_n16_ind_os,
                              return_n4_neg, return_n8_neg, return_n16_neg, return_n4_neg_os, 
                              return_n8_neg_os, return_n16_neg_os]

file_name = "DeepLearningResultsReturns.pkl"
open_file = open(file_name, "wb")
pickle.dump(DeepLearningResults_Returns, open_file)
open_file.close()

In [265]:
DeepLearningResults_Risks = [risks_n4_corr, risks_n8_corr, risks_n16_corr, risks_n4_corr_os, 
                              risks_n8_corr_os, risks_n16_corr_os, risks_n4_ind, risks_n8_ind,
                              risks_n16_ind, risks_n4_ind_os, risks_n8_ind_os, risks_n16_ind_os,
                              risks_n4_neg, risks_n8_neg, risks_n16_neg, risks_n4_neg_os, 
                              risks_n8_neg_os, risks_n16_neg_os]

file_name = "DeepLearningResultsRisks.pkl"
open_file = open(file_name, "wb")
pickle.dump(DeepLearningResults_Risks, open_file)
open_file.close()

## Archived

mae = np.zeros((99,1))
weight_n4_corr = pd.DataFrame(weight_n4_corr).dropna()
for i in range(99):
    mae[i] = sklearn.metrics.mean_absolute_error(np.array(weight_n4_corr)[i,:], [0.25, 0.25,0.25,0.25])
mae_5epoch = mse.mean()
mae_5epoch
np.array([mse_20epoch, mse_30epoch, mse_40epoch, mse_50epoch, mse_60epoch, mse_70epoch, mse_80epoch])
np.array([mae_20epoch, mae_30epoch, mae_40epoch, mae_50epoch, mae_60epoch])

def one_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications\
    '''
    weight_res = np.zeros((9,n_assets))
    #return_res = np.zeros((99,n_assets))
    #risks_res = np.zeros((99,n_assets))
    np.random.seed(32)
    
    for i in range(9):
        model = Model() 
        weights = model.get_allocations(pd.DataFrame(np.random.randn(n_assets,n_assets))) # what is this doing? This is kind of immportant to the model stability
        weight_res[i,:] = weights
        
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs)# gives nxk data 
        for j in range(n_assets):
            ret[j] = np.sum(wt[j] * return_vec[:,j])
            rsk[j] = wt[j] * np.diagonal(np.cov(returns_vec))[j] * wt[j]
        return_res[i,:] = returns.reshape(n_assets,)
        risks_res[i,:] = risks.reshape(n_assets,)
    return weight_res, return_res, risks_res